In [1]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.nn as nn
from PIL import Image
import torch.optim as optim
import torchvision
import torchvision.models as models
from torchvision import transforms
import time
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

import inspect
import matplotlib.pyplot as plt

import gc

gc.collect()

torch.cuda.empty_cache()

C:\Users\Asus\anaconda3\envs\venv_name\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Asus\anaconda3\envs\venv_name\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
class MnistResNet(nn.Module):
    def __init__(self,in_channels,n_classes):
        super(MnistResNet, self).__init__()

        self.model = models.resnet50(pretrained=True)

        self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Change the output layer to output 10 classes instead of 1000 classes
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, n_classes)

    def forward(self, x):
        return self.model(x)

class MnistVGG19(nn.Module):
    
    def __init__(self,in_channels,n_classes):
        super(MnistVGG19, self).__init__()
        self.model = models.vgg19(pretrained=True)
        in_features = self.model.classifier[-1].in_features
        self.model.classifier[-1] = nn.Linear(in_features, n_classes)

    def forward(self, x):
        return self.model(x)

    def forward(self, x):
        return self.model(x)

my_resnet = MnistResNet(3,2)
my_VGG16 = MnistVGG19(in_channels=3,n_classes=8)

In [3]:
input = torch.randn((16,3,244,244))
output = my_resnet(input)
print(output.shape)
print(my_resnet)
output = my_VGG16(input)
print(output.shape)
print(my_VGG16)

torch.Size([16, 2])
MnistResNet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): 

torch.Size([16, 8])
MnistVGG19(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inp

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

In [5]:
class Q1DataLoader():
    
    def __init__(self):
        self.data = np.load('../Assignment 1/data/pneumoniamnist.npz')
    
    def train_test_split(self):
        
        x_train = self.data['train_images']
        x_test = self.data['test_images']
        x_val = self.data['val_images']
        y_train = self.data['train_labels'].reshape((-1,))
        y_test = self.data['test_labels'].reshape((-1,))
        y_val = self.data['val_labels'].reshape((-1,))
        
        return x_train,y_train,x_test,y_test,x_val,y_val
    
    def get_metrics(self,pred,actual):
        
        n_correct_preds = 0
        tp = 0
        fp = 0
        tn = 0
        fn = 0
        for i in range(actual.shape[0]):
            if pred[i] == actual[i]:
                if actual[i]==1:
                    tp += 1
                else:
                    tn += 1
            else:
                if actual[i]==1:
                    fn += 1
                else:
                    fp += 1

        accuracy = (tp+tn)/(tp+fp+tn+fn)
        if tp+fn==0:
            recall= 0 
        else:
            recall = tp/(tp+fn)
        if tp+fp==0:
            precision = 0
        else:   
            precision = tp/(tp+fp)
        if recall==0 and precision==0:
            F1 = 0 
        else:
            F1 = 2*recall*precision/(recall+precision)
        if tn+fp==0:
            specificity = 0
        else:
            specificity = tn/(tn+fp)
        AUC = (recall + specificity)/2

        return accuracy,F1,AUC

In [6]:
class Q2DataLoader():
    
    def __init__(self):
        self.data = np.load('../Assignment 1/data/bloodmnist.npz')
    
    def train_test_split(self):
        
        x_train = self.data['train_images']
        x_test = self.data['test_images']
        x_val = self.data['val_images']
        y_train = self.data['train_labels'].reshape((-1,))
        y_test = self.data['test_labels'].reshape((-1,))
        y_val = self.data['val_labels'].reshape((-1,))
        return x_train,y_train,x_test,y_test,x_val,y_val
    
    def get_metrics(self,pred,actual):
        
        n_correct_preds = 0
        tp = 0
        fp = 0
        tn = 0
        fn = 0
        for i in range(actual.shape[0]):
            if pred[i] == actual[i]:
                if actual[i]==1:
                    tp += 1
                else:
                    tn += 1
            else:
                if actual[i]==1:
                    fn += 1
                else:
                    fp += 1

        accuracy = (tp+tn)/(tp+fp+tn+fn)
        if tp+fn==0:
            recall= 0 
        else:
            recall = tp/(tp+fn)
        if tp+fp==0:
            precision = 0
        else:   
            precision = tp/(tp+fp)
        if recall==0 and precision==0:
            F1 = 0 
        else:
            F1 = 2*recall*precision/(recall+precision)
        if tn+fp == 0:
            specificity = 0
        else:
            specificity = tn/(tn+fp)
        AUC = (recall + specificity)/2

        return accuracy,F1,AUC

In [10]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = torch.LongTensor(targets)
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        
        if self.transform:
            x = Image.fromarray(self.data[index].astype(np.uint8))
            x = self.transform(x)
        
        return x, y
    
    def __len__(self):
        return len(self.data)

train_x,train_y,val_x,val_y,test_x,test_y = Q2DataLoader().train_test_split()
print(train_x.shape,train_y.shape)
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5 ), (0.5,0.5,0.5))])
train_dataset = MyDataset(train_x, train_y, transform = transform)
val_dataset = MyDataset(val_x, val_y, transform = transform) 
test_dataset = MyDataset(test_x, test_y, transform = None) 
train_dataloader = DataLoader(train_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)
val_dataloader = DataLoader(val_dataset, batch_size=64)


(11959, 28, 28, 3) (11959,)


In [13]:
#CUDA_LAUNCH_BLOCKING=1

import gc
gc.collect()
torch.cuda.empty_cache()

#model = MnistResNet(1,2).to(device)
model = MnistVGG19(in_channels=3,n_classes=8).to(device)
# params you need to specify:
epochs = 10
batch_size = 64

# Dataloaders
#train_loader, val_loader = get_data_loaders(batch_size, batch_size)
train_dataloader = DataLoader(train_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)
val_dataloader = DataLoader(val_dataset, batch_size=64)

# loss function and optimizer
loss_function = nn.CrossEntropyLoss() # your loss function, cross entropy works well for multi-class problems

# optimizer, I've used Adadelta, as it wokrs well without any magic numbers
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4) 

start_ts = time.time()

losses = []
batches = len(train_dataloader)
val_batches = len(val_dataloader)
AUC, F1, accuracy = [],[],[]

# loop for every epoch (training + evaluation)
for epoch in range(epochs):
    total_loss = 0

    # progress bar (works in Jupyter notebook too!)
    progress = tqdm(enumerate(train_dataloader), desc="Loss: ", total=batches)

    # ----------------- TRAINING  -------------------- 
    # set model to training
    model.train()
    
    for i, data in progress:
        X, y = data[0].to(device), data[1].to(device)
        
        # training step for single batch
        model.zero_grad()
        outputs = model(X)
        loss = loss_function(outputs, y)
        loss.backward()
        optimizer.step()

        # getting training quality data
        current_loss = loss.item()
        total_loss += current_loss

        # updating progress bar
        progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
        
    # releasing unceseccary memory in GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # ----------------- VALIDATION  ----------------- 
    val_losses = 0
    AUC_l, f1_l, accuracy_l = [], [], []
    #p, r, f, a = [],[],[],[]
    # set model to evaluating (testing)
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_dataloader):
            X, y = data[0].to(device), data[1].to(device)

            outputs = model(X) # this get's the prediction from the network

            val_losses += loss_function(outputs, y)

            predicted_classes = torch.max(outputs, 1)[1] # get class from network's prediction
            
            a,f1,auc = Q1DataLoader().get_metrics(y.cpu(),predicted_classes.cpu())
            AUC_l.append(auc)
            f1_l.append(f1)
            accuracy_l.append(a)
          
    print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
    print(f"AUC = {sum(AUC_l)/len(AUC_l)},f1 = {sum(f1_l)/len(f1_l)},accuracy = {sum(accuracy_l)/len(accuracy_l)}")
    #print_scores(p, r, f, a, val_batches)
    AUC.append(sum(AUC_l)/len(AUC_l))
    accuracy.append(sum(accuracy_l)/len(accuracy_l))
    F1.append(sum(f1_l)/len(f1_l))
    losses.append(total_loss/batches) # for plotting learning curve
    
print(f"Training time: {time.time()-start_ts}s")
plt.plot(np.arange(0, len(losses)),losses)
plt.ylabel("Training Loss")
plt.xlabel("Steps")
plt.title("Loss versus Iterations")
plt.show()
plt.close()

plt.plot(np.arange(0, len(losses)),accuracy,label = "accuracy")
plt.plot(np.arange(0, len(losses)),F1,label = "F1")
plt.plot(np.arange(0, len(losses)),AUC,label = "AUC")
plt.ylabel("Metrics")
plt.xlabel("Steps")
plt.title("Metrics versus Iterations")
plt.legend()
plt.show()
plt.close()

Loss:   0%|                                                                                    | 0/187 [00:00<?, ?it/s]


RuntimeError: Given input size: (512x1x1). Calculated output size: (512x0x0). Output size is too small